# Example Kriding3D Project

## Step 1: Install required libraries

In [8]:
# conda install numpy pandas matplotlib scikit-learn pykrige skgstat streamlit rasterio pyproj tensorflow

## Step 2: Import required libraries

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skgstat import Variogram
from pykrige.ok3d import OrdinaryKriging3D
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
import streamlit as st
import rasterio
import pyproj
from tensorflow.keras import layers, models

## Step 3Load Data

In [10]:
# Load the dataset
data = pd.read_csv('df_merged_wells_sample.csv')

# Display the first few rows to inspect the data
print(data.head())

       API_WellNo  Cnty  Hole_x  SideTrck_x  Completion_x         Well_Name  \
0  31003659250000     3   65925           0             0    A&F Oil Co. 10   
1  31009661340000     9   66134           0             0           Kurty 7   
2  31003678950000     3   67895           0             0  J&R Oil Lot 8 22   
3  31003680490000     3   68049           0             0          Ewell 22   
4  31009224270000     9   22427           0             0        Yoder, A.2   

                        Company_name  Operator_number Well_Type Map_Symbol  \
0  Simon of Bolivar Enterprises Inc.             1335        OD         OW   
1                  Stephan Philip D.              566        OD         OW   
2               Sturdevant Walter B.             1483        OD         OW   
3       Sam Koehler Enterprises Inc.             2997        OD         OW   
4            Empire Energy E & P LLC             2544        GD         GW   

   ...          Field Wl_Status           Well_Nm      t

## Step 4: Data Preprocessing
1. Handle Missing Values: Check for missing values and decide on an appropriate strategy (drop or fill).
2. Outlier Detection and Handling: Identify and potentially handle outliers in the gas_production field.
3. Encoding Features and Feature Scaling: Scale features if necessary (e.g. categorical features transform with label encoding and: x, y, vertical_depth).

In [ ]:
# Check for missing values
print(data.isnull().sum())

# Drop rows with missing values (or fill with appropriate values if necessary)
data = data.dropna()

# Visualize outliers using a boxplot
plt.figure()
plt.boxplot(data['GasProd'])
plt.title('Boxplot for Gas Production')
plt.show()

# Optionally, handle outliers (e.g., clipping or removing them)
# data['gas_production'] = np.clip(data['gas_production'], lower_bound, upper_bound)
# Inspect the data types to identify categorical variables
print(data.dtypes)

# Assuming we have a categorical variable called 'category_column'
# Perform one-hot encoding for categorical variables

# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object', 'category']).columns

# One-hot encode the categorical columns
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Alternatively, if using label encoding
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Store the encoder for inverse_transform if needed

# Scale features
scaler = StandardScaler()
data[['x', 'y', 'vertical_depth']] = scaler.fit_transform(data[['x', 'y', 'vertical_depth']])

## Step 5: Split the Data into Training and Testing Sets

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

## Step 6: Compute the Variogram
We will calculate the variogram for the 3D coordinates (x, y, vertical_depth) against the target gas_production.

In [ ]:
coordinates = train[['x', 'y', 'vertical_depth']].values
values = train['gas_production'].values

# Create a 3D Variogram instance
variogram = Variogram(coordinates, values, model='spherical', n_lags=20)
variogram.plot()

## Step 7: Perform Ordinary 3D Kriging
Using the variogram, we'll perform 3D Kriging to predict gas production in the test data.

In [ ]:
# Set up the Ordinary Kriging 3D model
ok3d = OrdinaryKriging3D(
    train['x'].values, train['y'].values, train['vertical_depth'].values, train['gas_production'].values, 
    variogram_model='spherical'
)

# Get coordinates for prediction (from test set)
test_coordinates = test[['x', 'y', 'vertical_depth']].values

# Perform Kriging
predicted_values, variance = ok3d.execute('points', test['x'].values, test['y'].values, test['vertical_depth'].values)

## Step 8: Evaluate the Model
Let's calculate the Root Mean Squared Error (RMSE) to evaluate the performance of the Kriging model.

In [ ]:
# Add the predicted values to the test set
test['predicted_gas_production'] = predicted_values

# Calculate RMSE
rmse = np.sqrt(np.mean((test['gas_production'] - test['predicted_gas_production'])**2))
print(f'RMSE: {rmse:.4f}')

## Step 9: Create an Interpolation Grid
We'll create a grid over the 3D space of the wells to interpolate the gas production values.

In [ ]:
# Define grid for interpolation
grid_x, grid_y, grid_z = np.meshgrid(
    np.linspace(data['x'].min(), data['x'].max(), 50),
    np.linspace(data['y'].min(), data['y'].max(), 50),
    np.linspace(data['vertical_depth'].min(), data['vertical_depth'].max(), 50)
)

# Perform Kriging over the grid
interpolated_values, grid_variance = ok3d.execute('grid', grid_x, grid_y, grid_z)

## Step 10: Visualize the Results
We can visualize the interpolation results using 2D cross-sections from the 3D grid.

In [ ]:
# Visualizing a cross-section of the 3D Kriging result
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], interpolated_values[:, :, 25], cmap='viridis')
plt.colorbar(label='Interpolated Gas Production')
plt.title('Cross-section of Kriging Interpolation at Depth Level 25')

plt.subplot(1, 2, 2)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], grid_variance[:, :, 25], cmap='plasma')
plt.colorbar(label='Kriging Variance')
plt.title('Cross-section of Kriging Variance at Depth Level 25')

plt.show()


## Step 11: Save the Results
Save the interpolated values and variance for later use.

In [ ]:
# Save the interpolated values and variance
np.save('interpolated_values_3d.npy', interpolated_values)
np.save('grid_variance_3d.npy', grid_variance)

# Save plots as images
plt.savefig('kriging_3d_interpolation.png')
plt.savefig('kriging_3d_variance.png')


## Step 12: Documentation and Reporting
Prepare a detailed report that documents the following:

- Introduction: Overview of the project, objectives, and methodology.
- Data Description: Discuss the dataset, including preprocessing steps.
- Methodology: Explain the Kriging process, variogram modeling, and interpolation.
- Results: Present the RMSE, cross-validation results, and visualizations.
- Conclusion: Summarize the findings, discuss the implications, and suggest potential applications.

## Step 13: Cross-Validation and Model Validation
Perform cross-validation to evaluate the stability and generalizability of the Kriging model.

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []

for train_index, test_index in kf.split(data):
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
    coordinates_cv = train_cv[['x', 'y', 'vertical_depth']].values
    values_cv = train_cv['gas_production'].values
    
    variogram_cv = Variogram(coordinates_cv, values_cv, model='spherical')
    ok3d_cv = OrdinaryKriging3D(variogram_cv)
    
    test_coordinates_cv = test_cv[['x', 'y', 'vertical_depth']].values
    predicted_values_cv, _ = ok3d_cv.execute('points', test_cv['x'].values, test_cv['y'].values, test_cv['vertical_depth'].values)
    
    rmse_cv = np.sqrt(np.mean((test_cv['gas_production'] - predicted_values_cv)**2))
    rmse_scores.append(rmse_cv)

print(f'Cross-validated RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}')


## Step 14: Uncertainty Analysis
Analyze areas of high uncertainty in the Kriging model.

In [ ]:
# Identify areas of high uncertainty
high_uncertainty = grid_variance > np.percentile(grid_variance, 90)

plt.figure(figsize=(8, 6))
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], high_uncertainty[:, :, 25], cmap='Reds')
plt.colorbar(label='High Uncertainty Areas')
plt.title('High Uncertainty Areas in Kriging Prediction (Depth Level 25)')
plt.show()


## Step 15: Sensitivity Analysis
Perform a sensitivity analysis by testing different variogram models and their impact on the results.

In [ ]:
models = ['spherical', 'exponential', 'gaussian']
results = {}

for model in models:
    variogram = Variogram(coordinates, values, model=model)
    ok3d = OrdinaryKriging3D(variogram)
    interpolated_values, _ = ok3d.execute('grid', grid_x, grid_y, grid_z)
    
    # Store results
    results[model] = interpolated_values

# Plot to compare different variogram models
plt.figure(figsize=(18, 6))
for i, model in enumerate(models):
    plt.subplot(1, 3, i+1)
    plt.contourf(grid_x[:, :, 25], grid_y
    plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], results[model][:, :, 25], cmap='viridis')
    plt.colorbar(label=f'Interpolated Gas Production ({model})')
    plt.title(f'Kriging Interpolation - {model.capitalize()} (Depth Level 25)')
plt.show()


## Step 16: Parameter Tuning
Experiment with different parameters such as the number of lags (n_lags), variogram model, and Kriging parameters like min_points and max_points.

In [ ]:
# Example of parameter tuning
best_rmse = float('inf')
best_params = None

for model in ['spherical', 'exponential', 'gaussian']:
    for n_lags in [10, 20, 30]:
        variogram = Variogram(coordinates, values, model=model, n_lags=n_lags)
        ok3d = OrdinaryKriging3D(variogram)
        
        # Perform Kriging
        interpolated_values, _ = ok3d.execute('grid', grid_x, grid_y, grid_z)
        
        # Evaluate with test set
        predicted_values, _ = ok3d.execute('points', test['x'].values, test['y'].values, test['vertical_depth'].values)
        rmse = np.sqrt(np.mean((test['gas_production'] - predicted_values)**2))
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = (model, n_lags)

print(f'Best Model: {best_params[0]}, Best n_lags: {best_params[1]}, RMSE: {best_rmse:.4f}')


## Step 17: Comparison with Other Interpolation Methods
Compare the Kriging results with other interpolation methods such as 3D Inverse Distance Weighting (IDW) or Radial Basis Functions (RBF).

3D Inverse Distance Weighting (IDW) Example:

In [ ]:
from scipy.interpolate import griddata

# Use griddata for 3D IDW
interpolated_idw = griddata(
    coordinates, values, 
    (grid_x, grid_y, grid_z), 
    method='linear'
)

# Visualize a cross-section of the IDW result
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], interpolated_idw[:, :, 25], cmap='viridis')
plt.colorbar(label='Interpolated Gas Production (IDW)')
plt.title('Cross-section of IDW Interpolation at Depth Level 25')

plt.subplot(1, 2, 2)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], grid_variance[:, :, 25], cmap='plasma')
plt.colorbar(label='Kriging Variance')
plt.title('Cross-section of Kriging Variance at Depth Level 25')

plt.show()


Radial Basis Function (RBF) Example:

In [ ]:
from scipy.interpolate import Rbf

# Fit the RBF model
rbf = Rbf(train['x'], train['y'], train['vertical_depth'], train['gas_production'], function='linear')

# Interpolate the values
interpolated_rbf = rbf(grid_x, grid_y, grid_z)

# Visualize a cross-section of the RBF result
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], interpolated_rbf[:, :, 25], cmap='viridis')
plt.colorbar(label='Interpolated Gas Production (RBF)')
plt.title('Cross-section of RBF Interpolation at Depth Level 25')

plt.subplot(1, 2, 2)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], grid_variance[:, :, 25], cmap='plasma')
plt.colorbar(label='Kriging Variance')
plt.title('Cross-section of Kriging Variance at Depth Level 25')

plt.show()


## Step 18: Final Report and Presentation
Prepare a comprehensive report that includes:

- Introduction: Describe the purpose of the project, including the significance of predicting natural gas production.
- Data Description: Detail the dataset, including its source, features, and any preprocessing steps.
- Methodology: Discuss the variogram modeling, Kriging process, and comparison with other interpolation methods.
- Results: Present the RMSE values, cross-validation results, and visualizations for Kriging and other methods.
- Discussion: Analyze the results, including the uncertainty and sensitivity analysis.
- Conclusion: Summarize the findings and potential applications in the field.

## Step 19: Code Packaging and Automation
To facilitate reuse and automation, package your code into functions or classes:

In [ ]:
class Kriging3DModel:
    def __init__(self, data, variogram_model='spherical', n_lags=20, grid_res=50):
        self.data = data
        self.variogram_model = variogram_model
        self.n_lags = n_lags
        self.grid_res = grid_res
        self.coordinates = data[['x', 'y', 'vertical_depth']].values
        self.values = data['gas_production'].values

    def fit_variogram(self):
        self.variogram = Variogram(self.coordinates, self.values, model=self.variogram_model, n_lags=self.n_lags)
    
    def perform_kriging(self):
        self.ok3d = OrdinaryKriging3D(self.variogram)
        
        grid_x, grid_y, grid_z = np.meshgrid(
            np.linspace(self.data['x'].min(), self.data['x'].max(), self.grid_res),
            np.linspace(self.data['y'].min(), self.data['y'].max(), self.grid_res),
            np.linspace(self.data['vertical_depth'].min(), self.data['vertical_depth'].max(), self.grid_res)
        )
        
        self.interpolated_values, self.grid_variance = self.ok3d.execute('grid', grid_x, grid_y, grid_z)
    
    def evaluate(self, test_data):
        test_coordinates = test_data[['x', 'y', 'vertical_depth']].values
        predicted_values, _ = self.ok3d.execute('points', test_data['x'].values, test_data['y'].values, test_data['vertical_depth'].values)
        rmse = np.sqrt(np.mean((test_data['gas_production'] - predicted_values)**2))
        return rmse
    
    def save_results(self, filename):
        np.save(f'{filename}_interpolated_values.npy', self.interpolated_values)
        np.save(f'{filename}_grid_variance.npy', self.grid_variance)

# Example usage:
kriging_model = Kriging3DModel(data)
kriging_model.fit_variogram()
kriging_model.perform_kriging()
rmse = kriging_model.evaluate(test)
print(f'RMSE: {rmse:.4f}')
kriging_model.save_results('kriging_results')


## Step 20: Deployment and Application
Deploy the Kriging model within an oil and gas company's decision-making platform, or as part of a web-based application. This could involve:
- Building a Web Interface: Using frameworks like Flask or Django, create a web interface that allows users to input new well coordinates and get predictions.
- Integration with Existing Tools: Integrate with GIS tools or custom enterprise software for oil and gas operations.

## Step 21: Advanced Variogram Modeling
Explore more advanced variogram models such as:
- Nested Variograms: Combine multiple variogram models to capture complex spatial patterns.
- Anisotropic Variograms: Model directional dependence by fitting anisotropic variograms.

In [ ]:
# Step 21 Example: Anisotropic variogram
variogram = Variogram(coordinates, values, model='spherical', anisotropy_scaling=2.0, anisotropy_angle=45)
variogram.plot()

## Step 22: Conditional Simulation
Generate multiple realizations of the gas production field to assess uncertainty in predictions.

In [ ]:
from pykrige.ok3d import OrdinaryKriging3D

# Perform conditional simulation (generating multiple realizations)
simulations = []
n_simulations = 100
for _ in range(n_simulations):
    ok3d = OrdinaryKriging3D(variogram)
    sim_values, _ = ok3d.execute('grid', grid_x, grid_y, grid_z, pseudo_data=True)
    simulations.append(sim_values)

# Calculate the mean and standard deviation of the simulations
simulated_mean = np.mean(simulations, axis=0)
simulated_std = np.std(simulations, axis=0)

# Visualize the simulated mean and standard deviation
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], simulated_mean[:, :, 25], cmap='viridis')
plt.colorbar(label='Simulated Mean Gas Production')
plt.title('Simulated Mean (Depth Level 25)')

plt.subplot(1, 2, 2)
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], simulated_std[:, :, 25], cmap='plasma')
plt.colorbar(label='Simulated Std Dev of Gas Production')
plt.title('Simulated Std Dev (Depth Level 25)')
plt.show()


## Step 23: Multivariate Kriging (Co-Kriging)
If you have additional related variables, you can use Co-Kriging to leverage the spatial correlation between variables. Co-Kriging requires extending the variogram to include cross-covariance between the primary and secondary variables.

Example of Co-Kriging:
Assume we have another variable, such as oil_production, that could be spatially correlated with gas_production.

In [ ]:
# Prepare data for Co-Kriging
secondary_variable = train['oil_production'].values  # Assume oil_production is a column in the dataset

# Use PyKrige's Co-Kriging capabilities
from pykrige.ok import UniversalKriging

# Fit a 3D Co-Kriging model
uk3d = UniversalKriging(
    train['x'].values, train['y'].values, train['vertical_depth'].values, train['gas_production'].values, 
    variogram_model='spherical', drift_terms=['regional_linear']
)

# Predict gas production using the Co-Kriging model
predicted_gas, variance = uk3d.execute('grid', grid_x, grid_y, grid_z)

# Visualize the results
plt.figure(figsize=(10, 6))
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], predicted_gas[:, :, 25], cmap='viridis')
plt.colorbar(label='Co-Kriged Gas Production')
plt.title('Co-Kriging Interpolation at Depth Level 25')
plt.show()


## Step 24: Geostatistical Uncertainty Quantification
Quantify the total uncertainty in the model predictions, incorporating both the Kriging variance and potential model uncertainties (e.g., from the variogram model).

In [ ]:
# Calculate total uncertainty
total_uncertainty = np.sqrt(grid_variance + simulated_std**2)

# Visualize the total uncertainty
plt.figure(figsize=(8, 6))
plt.contourf(grid_x[:, :, 25], grid_y[:, :, 25], total_uncertainty[:, :, 25], cmap='plasma')
plt.colorbar(label='Total Uncertainty')
plt.title('Total Uncertainty in Gas Production Prediction (Depth Level 25)')
plt.show()


## Step 25: Model Diagnostics
Perform detailed diagnostics on the Kriging model to ensure it meets the assumptions:
- Normality Check: Verify that the residuals are normally distributed.
- Homoscedasticity Check: Ensure the residual variance is constant across predicted values.

In [ ]:
# Residuals analysis
residuals = test['gas_production'] - test['predicted_gas_production']

# Normality check
plt.figure()
plt.hist(residuals, bins=20, color='gray', edgecolor='black')
plt.title('Histogram of Residuals')
plt.show()

# Residuals vs Predicted values
plt.figure()
plt.scatter(test['predicted_gas_production'], residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted Gas Production')
plt.xlabel('Predicted Gas Production')
plt.ylabel('Residuals')
plt.show()


## Step 26: Spatial Model Validation
Beyond just evaluating the RMSE, compare the Kriging model with other spatial models. Consider models such as Bayesian Kriging, Gaussian Process Regression, or even spatial machine learning models (e.g., Spatial Random Forest).

Example of Bayesian Kriging:

In [ ]:
from pykrige.uk import UniversalKriging3D

# Set up Bayesian Universal Kriging
uk3d = UniversalKriging3D(
    train['x'].values, train['y'].values, train['vertical_depth'].values, train['gas_production'].values,
    variogram_model='spherical', drift_terms=['regional_linear']
)

# Predict using Bayesian Kriging
predicted_values_uk, variance_uk = uk3d.execute('points', test['x'].values, test['y'].values, test['vertical_depth'].values)

# Calculate RMSE
rmse_uk = np.sqrt(np.mean((test['gas_production'] - predicted_values_uk)**2))
print(f'RMSE (Bayesian Kriging): {rmse_uk:.4f}')


## Step 27: Sensitivity to Data Quality
Test how the model performs under varying data quality conditions, such as introducing noise or reducing the number of data points.

Example: Adding Noise to the Data

In [ ]:
# Add random noise to the gas production values
noise_level = 0.1  # 10% noise
noisy_data = data.copy()
noisy_data['gas_production'] += noise_level * np.random.normal(size=data.shape[0])

# Perform Kriging on noisy data
kriging_model_noisy = Kriging3DModel(noisy_data)
kriging_model_noisy.fit_variogram()
kriging_model_noisy.perform_kriging()
rmse_noisy = kriging_model_noisy.evaluate(test)
print(f'RMSE with Noisy Data: {rmse_noisy:.4f}')


## Step 28: Real-World Application and Decision-Making
In the context of oil and gas operations, use the Kriging model to guide decisions such as:
- Well Placement: Identify optimal locations for drilling new wells.
- Resource Estimation: Estimate total recoverable gas reserves in a field.
- Risk Management: Use uncertainty quantification to manage risks in exploration and production.

## Step 29: Deployment and Scalability
If the model needs to be applied at scale (e.g., across a large oil field with millions of data points), consider:
- Parallel Computing: Distribute computations across multiple processors to handle large datasets.
- Cloud Deployment: Deploy the model on cloud platforms (e.g., AWS, Google Cloud) to scale with computational resources.

## Step 30: Final Report with Policy Implications
If the results of your Kriging model are used to inform policy or business strategy, the final report should include:
- Clear Recommendations: Based on the model’s predictions, recommend specific actions or policies.
- Ethical Considerations: Discuss any ethical implications, such as the impact of drilling on local communities or the environment.
- Stakeholder Engagement: Include input from key stakeholders to ensure the model’s predictions align with broader organizational goals.

## Step 31: Bayesian Kriging
Implement Bayesian Kriging, which incorporates prior distributions for the variogram parameters and generates a full posterior distribution for the predictions.

Example Using PyMC3 (a Bayesian library):

In [ ]:
import pymc3 as pm

# Define the model
with pm.Model() as kriging_model:
    # Priors for variogram parameters
    range_ = pm.Normal('range', mu=50, sigma=10)
    sill = pm.Normal('sill', mu=1, sigma=0.5)
    nugget = pm.Normal('nugget', mu=0.1, sigma=0.05)
    
    # Define the variogram model
    variogram = pm.Deterministic('variogram', sill * (1.0 - pm.math.exp(-3.0 * (coordinates / range_)**2)) + nugget)
    
    # Likelihood
    observed_data = pm.Normal('observed_data', mu=variogram, sigma=sill, observed=values)
    
    # Sample from the posterior
    trace = pm.sample(1000, return_inferencedata=True)

# Summary of the posterior
pm.summary(trace)


## Step 32: Integration with Geographic Information Systems (GIS)
Export your 3D Kriging results to GIS formats like GeoTIFF or Shapefiles for integration with GIS tools such as QGIS or ArcGIS.

In [ ]:
import rasterio
from rasterio.transform import from_origin

# Exporting interpolated values to a GeoTIFF file
with rasterio.open(
    'kriging_3d_interpolation.tif', 'w',
    driver='GTiff',
    height=interpolated_values.shape[0],
    width=interpolated_values.shape[1],
    count=1,
    dtype=str(interpolated_values.dtype),
    crs='+proj=latlong',
    transform=from_origin(data['x'].min(), data['y'].max(), 1, 1)
) as dst:
    dst.write(interpolated_values[:, :, 25], 1)


## Step 33: Dynamic Kriging for Spatio-Temporal Data
Extend your model to handle data that varies over time (e.g., monthly gas production). This involves incorporating time as a fourth dimension in the Kriging process.

In [ ]:
# Include time as a fourth dimension
data['time'] = pd.to_datetime(data['date']).astype(int)  # Assuming 'date' is a column in the dataset

# 4D Kriging with time as the fourth dimension
coordinates_4d = data[['x', 'y', 'vertical_depth', 'time']].values
variogram_4d = Variogram(coordinates_4d, values, model='spherical')
ok4d = OrdinaryKriging3D(variogram_4d)

# Perform 4D Kriging
grid_x, grid_y, grid_z, grid_t = np.meshgrid(
    np.linspace(data['x'].min(), data['x'].max(), 50),
    np.linspace(data['y'].min(), data['y'].max(), 50),
    np.linspace(data['vertical_depth'].min(), data['vertical_depth'].max(), 50),
    np.linspace(data['time'].min(), data['time'].max(), 50)
)
interpolated_values_4d, grid_variance_4d = ok4d.execute('grid', grid_x, grid_y, grid_z, grid_t)


## Step 34: Machine Learning-Augmented Kriging
Combine Kriging with machine learning models to improve predictions, especially in capturing non-linear relationships.

Example: Combining Kriging with Random Forest:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

# Train a Random Forest model on the data
rf_model = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100))
rf_model.fit(train[['x', 'y', 'vertical_depth']], train['gas_production'])

# Use Kriging predictions as features in the Random Forest
kriging_features = ok3d.execute('points', train['x'].values, train['y'].values, train['vertical_depth'].values)[0]
train['kriging_pred'] = kriging_features

# Retrain the Random Forest model using Kriging predictions as an additional feature
rf_model.fit(train[['x', 'y', 'vertical_depth', 'kriging_pred']], train['gas_production'])

# Predict using the hybrid model
kriging_features_test = ok3d.execute('points', test['x'].values, test['y'].values, test['vertical_depth'].values)[0]
test['kriging_pred'] = kriging_features_test
rf_predictions = rf_model.predict(test[['x', 'y', 'vertical_depth', 'kriging_pred']])

# Evaluate the hybrid model
rmse_rf_kriging = np.sqrt(np.mean((test['gas_production'] - rf_predictions.ravel())**2))
print(f'RMSE (Random Forest + Kriging): {rmse_rf_kriging:.4f}')


## Step 35: Automated Kriging Workflows
Automate the process of selecting the best variogram model, tuning parameters, and performing validation using tools like AutoML or custom automation scripts.

In [ ]:
# Example: Automating variogram selection and Kriging process
best_rmse = float('inf')
best_params = None
for model in ['spherical', 'exponential', 'gaussian']:
    for n_lags in [10, 20, 30]:
        variogram = Variogram(coordinates, values, model=model, n_lags=n_lags)
        ok3d = OrdinaryKriging3D(variogram)
        
        # Perform Kriging
        interpolated_values, _ = ok3d.execute('grid', grid_x, grid_y, grid_z)
        
        # Evaluate with test set
        predicted_values, _ = ok3d.execute('points', test['x'].values, test['y'].values, test['vertical_depth'].values)
        rmse = np.sqrt(np.mean((test['gas_production'] - predicted_values)**2))
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = (model, n_lags)

print(f'Best Model: {best_params[0]}, Best n_lags: {best_params[1]}, RMSE: {best_rmse:.4f}')


## Step 36: 3D Kriging
This step focuses on implementing 3D Kriging from the beginning, accounting for the vertical depth in the well data. This was covered in earlier steps, but here’s a quick recap:

In [ ]:
# 3D Kriging was covered in earlier steps; here's a quick summary of the process:
ok3d = OrdinaryKriging3D(
    train['x'].values, train['y'].values, train['vertical_depth'].values, train['gas_production'].values, 
    variogram_model='spherical'
)
grid_x, grid_y, grid_z = np.meshgrid(
    np.linspace(data['x'].min(), data['x'].max(), 50),
    np.linspace(data['y'].min(), data['y'].max(), 50),
    np.linspace(data['vertical_depth'].min(), data['vertical_depth'].max(), 50)
)
interpolated_values, grid_variance = ok3d.execute('grid', grid_x, grid_y, grid_z)


## Step 37: Global Kriging
Global Kriging extends the model to work over a larger geographic area, taking into account the curvature of the Earth. This requires using spherical coordinates and potentially modifying the variogram model to accommodate large-scale non-stationarity.

Steps for Global Kriging:
- Convert Coordinates: Transform the x, y coordinates into latitude and longitude or other global coordinate systems.
- Adjust the Variogram: Modify the variogram to handle larger geographic scales.
- Perform Kriging: Use spherical coordinates in the Kriging process.

In [ ]:
# Transforming coordinates for global scale (example using lat/long)
# Assuming 'longitude' and 'latitude' are columns in the dataset
import pyproj

proj = pyproj.Proj(proj='latlong', datum='WGS84')
data['x'], data['y'] = proj(data['longitude'].values, data['latitude'].values)

# Use these transformed coordinates in the Kriging process
coordinates_global = data[['x', 'y', 'vertical_depth']].values
variogram_global = Variogram(coordinates_global, values, model='spherical')
ok3d_global = OrdinaryKriging3D(variogram_global)

# Perform global Kriging
interpolated_values_global, grid_variance_global = ok3d_global.execute('grid', grid_x, grid_y, grid_z)


## Step 38: Deep Learning-Based Spatial Modeling
Integrate deep learning approaches, such as Convolutional Neural Networks (CNNs), with Kriging for spatial modeling. This can be particularly useful for capturing complex, non-linear spatial patterns.

Example Using a CNN for Spatial Data:

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Predict gas production
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Reshape data for CNN input
X_train = interpolated_values[:, :, 25].reshape(-1, 50, 50, 1)  # Example for a 2D cross-section
y_train = train['gas_production'].values

# Train the model
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32)

# Use the CNN model to predict gas production on the test set
X_test = grid_x[:, :, 25].reshape(-1, 50, 50, 1)
cnn_predictions = cnn_model.predict(X_test)

# Evaluate the CNN model
rmse_cnn = np.sqrt(np.mean((test['gas_production'] - cnn_predictions.ravel())**2))
print(f'RMSE (CNN): {rmse_cnn:.4f}')


## Step 39: Ethics and Fairness in Spatial Predictions
Evaluate the ethical implications of your predictions, particularly if they affect communities, environments, or resource management. Consider:

- Bias: Ensure that the model doesn't introduce bias that could disproportionately affect certain groups.
- Transparency: Clearly communicate the limitations and uncertainties of your model to stakeholders.
- Sustainability: Assess the long-term environmental impacts of decisions based on your model.

## Step 40: Interactive Web Applications
Develop an interactive web application that allows users to explore Kriging predictions dynamically. This could be built using frameworks like Flask, Django, or Streamlit.

Example Using Streamlit:

In [ ]:
import streamlit as st

# Build an interactive app with Streamlit
st.title("3D Kriging for Gas Production Prediction")

# Input fields for new well coordinates
x_input = st.number_input("Enter x-coordinate", value=0.0)
y_input = st.number_input("Enter y-coordinate", value=0.0)
depth_input = st.number_input("Enter vertical depth", value=0.0)

# Predict gas production for the input coordinates
predicted_gas, _ = ok3d.execute('points', np.array([x_input]), np.array([y_input]), np.array([depth_input]))
st.write(f"Predicted Gas Production: {predicted_gas[0]:.2f}")

# Option to visualize the entire 3D grid
if st.button("Show 3D Kriging Interpolation"):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    ax.scatter(grid_x, grid_y, grid_z, c=interpolated_values.ravel(), cmap='viridis')
    st.pyplot(fig)


This web application allows users to input coordinates for new wells and instantly see predicted gas production values based on the Kriging model. The app can be extended with additional features such as sensitivity analysis, uncertainty visualization, and more.







## Script

In [ ]:
try:
    print('Script executed successfully.')
except:
    print('FAILED')